In [ ]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [ ]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math

In [ ]:
DATA_PATH = "predictions/preds_next_season.csv"

In [ ]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [ ]:
df.columns

In [ ]:
df = df[df["GW"] == 3]

In [ ]:
df.head(30)

In [ ]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [ ]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

In [ ]:
df.head()

In [ ]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [ ]:

for p in playersDB:
    print(p.__str__())

In [ ]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [ ]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 3, 2],
    [5, 4, 1],
]

In [ ]:
budget = 1000

In [ ]:
%%time
best_team = best_full_teams(playersDB, possible_formations, budget)

In [ ]:
f"Best team score: {best_team[0][1]}"

In [ ]:
def best_team_to_df(best_team_squad, df, custom_fpl_order):
    names = [p.name for p in best_team_squad]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    return df[df["Name_original"].isin(names)].sort_values(by=["FPL_pos", "Price"], ascending=False)

# Define your custom order for FPL_pos
custom_fpl_order = ["FWD", "MID", "DEF", "GK"]

In [ ]:
best_squad = best_team_to_df(best_team[0][2], df, custom_fpl_order)
best_squad

In [ ]:
best_squad.to_csv("predictions/best_squad.csv")